In [4]:
from src.llm import *
from datasets import load_dataset

CONFIG = {
    "llm_model_name": "Salesforce/xLAM-2-1b-fc-r",
    "dataset": "Salesforce/xlam-function-calling-60k",
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}
llm_model, llm_tokenizer = load_llm(CONFIG)

ds = load_dataset(CONFIG["dataset"])

In [7]:
sample = ds["train"][0]
tools = json.loads(sample['tools']) if isinstance(sample['tools'], str) else sample['tools']

# Create the structured output processor
processor = StructuredFunctionCallProcessor(
    tokenizer=llm_tokenizer,
    tools=tools,
    device=llm_model.device
)

In [9]:
import json
with open("/workspace/removed_sample_indices.json", "r") as f:
    removed_sample_indices = json.load(f)
for i in removed_sample_indices["removed_indices"][:10]:
    print(ds["train"][i])


{'id': 32, 'query': 'Project the investment growth for an initial amount of $10000, with an annual addition of $1000, over 5 years at a 5% return rate, with inflation rates of [2%, 2.5%, 3%, 3.5%, 4%].', 'answers': '[{"name": "project_investment_growth", "arguments": {"principal": 10000, "annual_addition": 1000, "years": 5, "return_rate": 0.05, "inflation": [0.02, 0.025, 0.03, 0.035, 0.04], "inflation_adjusted": true}}]', 'tools': '[{"name": "get_dna_sequence", "description": "Retrieves the DNA sequence for a given sequence ID from the NCBI Nucleotide database.", "parameters": {"sequence_id": {"description": "The unique identifier for the DNA sequence.", "type": "str", "default": "fasta"}, "file_format": {"description": "The format of the returned sequence. Allowed values: \\"fasta\\" (default) or \\"gb\\".", "type": "str, optional"}, "upstream_bases": {"description": "The number of bases upstream of the sequence to include. Defaults to 0.", "type": "int, optional", "default": "fasta"}

In [11]:
from datasets import load_from_disk
ds = load_from_disk("/workspace/xlam-function-calling-60k-filtered")
ds[0]

{'id': 0,
 'query': 'Where can I find live giveaways for beta access and games?',
 'answers': '[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]',
 'tools': '[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]'}

In [5]:
generate_structured_function_calls(ds["train"][0], llm_model, llm_tokenizer)

'    ]   ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]  ]'

In [ ]:
logits_processor = StructuredFunctionCallProcessor(
    tokenizer=llm_tokenizer,
    tools=ds["train"][0]["input_ids"],
    device=CONFIG["device"],
)

